In [2]:
import numpy as np
from scipy.optimize import leastsq
import math
from utils import *

In [3]:
def calc_matrix(params):
    theta__7, r__7, theta__8, r__8, alpha__8, d__8 = params
    return np.mat([[math.cos(theta__7) * math.cos(theta__8) - math.sin(theta__7) * math.cos(alpha__8) * math.sin(theta__8),-math.cos(theta__7) * math.sin(theta__8) - math.sin(theta__7) * math.cos(alpha__8) * math.cos(theta__8),math.sin(theta__7) * math.sin(alpha__8),math.sin(theta__7) * math.sin(alpha__8) * r__8 + math.cos(theta__7) * d__8],[math.sin(theta__7) * math.cos(theta__8) + math.cos(theta__7) * math.cos(alpha__8) * math.sin(theta__8),math.cos(theta__7) * math.cos(alpha__8) * math.cos(theta__8) - math.sin(theta__7) * math.sin(theta__8),-math.cos(theta__7) * math.sin(alpha__8),-math.cos(theta__7) * math.sin(alpha__8) * r__8 + d__8 * math.sin(theta__7)],[math.sin(alpha__8) * math.sin(theta__8),math.sin(alpha__8) * math.cos(theta__8),math.cos(alpha__8),math.cos(alpha__8) * r__8 + r__7],[0,0,0,1]])


In [4]:
def eq(params, res):
    mat = np.array(calc_matrix(params)-res)
    return mat.flatten()

In [5]:
def reduce_rotations(theta):
    if theta > np.pi:
        theta = theta % (2*np.pi)
    if theta < -np.pi:
        theta = theta % -(2*np.pi)
    return theta

In [6]:
def find_mdh_params_6d(mat, initial_guesses = (0, 0, 0, 0, 0, 0), verbose=False):
    if verbose:
        print(f"Target Matrix:\n")
        roundprint(np.array(mat))
    res = leastsq(eq, initial_guesses, args=mat)[0]
    
    
    if False:
        results = zip(["theta__7", "r__7", "theta__8", "r__8", "alpha__8", "d__8" ], res)
        print(f"Results:")
        for elem in results:
            print("\t"+elem[0]+" :\t\t"+f"{elem[1]:.2f}")
        print(f"\nCheck Matrix:")
        roundprint(np.array(calc_matrix(tuple(res))))
        
    # modulo for the angles
    res2 = res.copy()
    res2[0] = reduce_rotations(res2[0])
    res2[2] = reduce_rotations(res2[2])
    res2[4] = reduce_rotations(res2[4])
    
    if verbose:
        results = zip(["theta__7", "r__7", "theta__8", "r__8", "alpha__8", "d__8" ], res2)
        print(f"Results Modulo:")
        for elem in results:
            print("\t"+elem[0]+" :\t\t"+f"{elem[1]:.2f}")
        print(f"\nCheck Matrix Modulo:")
        roundprint(np.array(calc_matrix(tuple(res2))))
    print("residue")
    roundprint(np.array(calc_matrix(tuple(res2)))-mat)
    return res2

In [7]:
# random input matrix from angle axis
rvec = np.random.uniform(low=0.0, high=1, size=(3,))*np.random.uniform(low=0.0, high=np.pi, size=(1,))
tvec = np.random.uniform(low=0.0, high=0.2, size=(3,))
print(rvec, tvec)
target_matrix = H_rvec_tvec(rvec, tvec)

# find MDH values
res = find_mdh_params_6d(target_matrix, verbose=True)

[0.32519531 0.95113805 0.68051671] [0.06549159 0.08273894 0.08831115]
Target Matrix:

 0.396 -0.389  0.832  0.065 
 0.662  0.749  0.035  0.083 
-0.636  0.537  0.554  0.088 
 0.000  0.000  0.000  1.000 


Results Modulo:
	theta__7 :		-4.67
	r__7 :		0.04
	theta__8 :		5.41
	r__8 :		0.08
	alpha__8 :		0.98
	d__8 :		0.08

Check Matrix Modulo:
 0.396 -0.389  0.832  0.065 
 0.662  0.749  0.035  0.083 
-0.636  0.537  0.554  0.088 
 0.000  0.000  0.000  1.000 


residue
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 




In [8]:
#input matrix 
target_matrix = Trans(0, 0, 0.2)@Rx(np.pi/2)
# find MDH values
res = find_mdh_params_6d(target_matrix, verbose=True)

Target Matrix:

 1.000  0.000  0.000  0.000 
 0.000  0.000 -1.000  0.000 
 0.000  1.000  0.000  0.200 
 0.000  0.000  0.000  1.000 


Results Modulo:
	theta__7 :		-0.00
	r__7 :		0.20
	theta__8 :		-0.00
	r__8 :		0.00
	alpha__8 :		1.57
	d__8 :		-0.00

Check Matrix Modulo:
 1.000  0.000  0.000  0.000 
 0.000  0.000 -1.000  0.000 
 0.000  1.000  0.000  0.200 
 0.000  0.000  0.000  1.000 


residue
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 




In [27]:
def rpy_trans_to_mat(roll, pitch, yaw, x, y, z):
    print(f"rpy:{[roll, pitch, yaw]}")
    print(f"xyz:{[x, y, z]}")
    print("\n")
    T = Trans(x,y,z)@Rz(yaw)@Ry(pitch)@Rx(roll)
    return T


In [32]:
# random input 
# v1 = np.random.uniform(low=0.0, high=1, size=(3,))
# v2 = np.random.uniform(low=0.0, high=0.2, size=(3,))
# rvec=np.round(rvec,2)
# tvec=np.round(tvec,2)

target_matrix = rpy_trans_to_mat(1, 0, 0, 0.1, 0, 0.3)

# find MDH values
res = find_mdh_params_6d(target_matrix, verbose=True)

rpy:[1, 0, 0]
xyz:[0.1, 0, 0.3]


Target Matrix:

 1.000  0.000  0.000  0.100 
 0.000  0.540 -0.841  0.000 
 0.000  0.841  0.540  0.300 
 0.000  0.000  0.000  1.000 


Results Modulo:
	theta__7 :		0.00
	r__7 :		0.30
	theta__8 :		-0.00
	r__8 :		0.00
	alpha__8 :		1.00
	d__8 :		0.10

Check Matrix Modulo:
 1.000  0.000  0.000  0.100 
 0.000  0.540 -0.841  0.000 
 0.000  0.841  0.540  0.300 
 0.000  0.000  0.000  1.000 


residue
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 
 0.000  0.000  0.000  0.000 


